In [1]:
from transformers import AutoTokenizer,AutoModelForCausalLM
import torch
from torch.utils.data import Dataset,DataLoader
from torch.nn.utils.rnn import pad_sequence

In [2]:
tokenizer=AutoTokenizer.from_pretrained("gpt2")
model=AutoModelForCausalLM.from_pretrained("gpt2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [3]:
num_params = sum(p.numel() for p in model.parameters())

print(f"Total number of parameters: {num_params}")

Total number of parameters: 124439808


In [4]:
prompt="Dear boss..."
input_ids=tokenizer.encode(prompt,return_tensors="pt")
outputs=model.generate(input_ids,max_length=100)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [5]:
eng=tokenizer.decode(outputs[0],skip_special_tokens=True)
eng

"Dear boss... I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I"

In [6]:
# Simplified text generation function
def simple_text_generation(prompt, model, tokenizer, max_length = 100):
  # Encoding the prompt to get the input ids
  input_ids = tokenizer.encode(prompt, return_tensors="pt") # pt = pytorch

  # Generate text using the model
  outputs = model.generate(input_ids, max_length = 100)

  # Decode the generated output IDs back into text
  return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [7]:
prompt="Dear Boss,"
text_generated = simple_text_generation(prompt,
                                        model,
                                        tokenizer,
                                        max_length = 100)
print(text_generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Dear Boss, I'm not going to tell you how to do it. I'm not going to tell you how to do it. I'm not going to tell you how to do it. I'm not going to tell you how to do it. I'm not going to tell you how to do it. I'm not going to tell you how to do it. I'm not going to tell you how to do it. I'm not going to tell you how to do it. I


# Finetuning GPT2

In [8]:
# Load dataset (scientific research abstracts related to machine learning)
data = [
    "This paper presents a new method for improving the performance of machine learning models by using data augmentation techniques.",
    "We propose a novel approach to natural language processing that leverages the power of transformers and attention mechanisms.",
    "In this study, we investigate the impact of deep learning algorithms on the accuracy of image recognition tasks.",
    "Our research demonstrates the effectiveness of transfer learning in enhancing the capabilities of neural networks.",
    "This work explores the use of reinforcement learning for optimizing decision-making processes in complex environments.",
    "We introduce a framework for unsupervised learning that significantly reduces the need for labeled data.",
    "The results of our experiments show that ensemble methods can substantially boost model performance.",
    "We analyze the scalability of various machine learning algorithms when applied to large datasets.",
    "Our findings suggest that hyperparameter tuning is crucial for achieving optimal results in machine learning applications.",
    "This research highlights the importance of feature engineering in the context of predictive modeling."
]

In [9]:
tokenizer.pad_token=tokenizer.eos_token

In [10]:
tokenized_data=[tokenizer.encode_plus(
    sentence,add_special_tokens=True,
    return_tensors="pt",
    padding="max_length",
    max_length=50) for sentence in data]
tokenized_data[:2]

[{'input_ids': tensor([[ 1212,  3348, 10969,   257,   649,  2446,   329, 10068,   262,  2854,
            286,  4572,  4673,  4981,   416,  1262,  1366, 16339, 14374,  7605,
             13, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0]])},
 {'input_ids': tensor([[ 1135, 18077,   257,  5337,  3164,   284,  3288,  3303,  7587,   326,
          17124,  1095,   262,  1176,   286,  6121,   364,   290,  3241, 11701,
             13, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50256, 50256, 50256, 502

In [11]:
# Isolate the input IDs and the attention masks
inputs_ids = [item['input_ids'].squeeze() for item in tokenized_data]
attention_masks = [item['attention_mask'].squeeze() for item in tokenized_data]
attention_masks[:2]

[tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0])]

In [12]:
# Convert the input IDs and attention masks to tensors
# This step is necessary for processing the tuned model
inputs_ids = torch.stack(inputs_ids)
attention_masks = torch.stack(attention_masks)

In [13]:
# Padding all input sequences to ensure they have the same length
padded_input_ids = pad_sequence(
    inputs_ids,
    batch_first = True,
    padding_value = tokenizer.eos_token_id) # Use the tokenizer's end-of-sequence token ID as the padding value

# Padding all attention masks to ensure they have the same length
padded_attention_masks = pad_sequence(
    attention_masks,
    batch_first = True,
    padding_value = 0) # Use 0 as the padding value for attention masks

In [14]:
# Create a custom dataset class for handling text data, including input IDs and attention masks
class TextDataset(Dataset):
  def __init__(self, input_ids, attention_masks):
    self.input_ids = input_ids  # Store the input IDs
    self.attention_masks = attention_masks  # Store the attention masks
    self.labels = input_ids.clone()  # Create labels identical to input IDs for tasks like language modeling

  def __len__(self):
    return len(self.input_ids)  # Return the number of samples in the dataset

  def __getitem__(self, idx):
    # Return a dictionary containing the input IDs, attention mask, and labels for a given index
    return {
        'input_ids': self.input_ids[idx],
        'attention_mask': self.attention_masks[idx],
        'labels': self.labels[idx]
    }

# Instantiate the dataset using the padded input IDs and attention masks
dataset = TextDataset(padded_input_ids, padded_attention_masks)
dataset[:2]

{'input_ids': tensor([[ 1212,  3348, 10969,   257,   649,  2446,   329, 10068,   262,  2854,
            286,  4572,  4673,  4981,   416,  1262,  1366, 16339, 14374,  7605,
             13, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256],
         [ 1135, 18077,   257,  5337,  3164,   284,  3288,  3303,  7587,   326,
          17124,  1095,   262,  1176,   286,  6121,   364,   290,  3241, 11701,
             13, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256]]),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
 

In [15]:
dataloader=DataLoader(dataset,batch_size=2,shuffle=True)

In [16]:
optimizer=torch.optim.Adam(model.parameters(),lr=5e-5)

In [17]:
model.train()
for epoch in range(10):
  for batch in dataloader:
    input_ids=batch["input_ids"]
    attention_mask=batch["attention_mask"]
    optimizer.zero_grad()
    outputs=model(input_ids=input_ids,attention_mask=attention_mask,labels=input_ids)
    loss=outputs.loss
    loss.backward()
    optimizer.step()
    print(f"epoch {epoch} --- loss {loss}")

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


epoch 0 --- loss 7.852370262145996
epoch 0 --- loss 6.388238430023193
epoch 0 --- loss 3.796618700027466
epoch 0 --- loss 2.14255952835083
epoch 0 --- loss 1.465640902519226
epoch 1 --- loss 1.3086671829223633
epoch 1 --- loss 1.2925567626953125
epoch 1 --- loss 1.2661699056625366
epoch 1 --- loss 1.1122455596923828
epoch 1 --- loss 1.4186086654663086
epoch 2 --- loss 1.0583062171936035
epoch 2 --- loss 1.0558979511260986
epoch 2 --- loss 1.086638331413269
epoch 2 --- loss 0.9687449932098389
epoch 2 --- loss 0.8908012509346008
epoch 3 --- loss 0.8612986207008362
epoch 3 --- loss 0.9059723019599915
epoch 3 --- loss 0.9465696215629578
epoch 3 --- loss 0.9636932611465454
epoch 3 --- loss 0.7807286977767944
epoch 4 --- loss 0.77545565366745
epoch 4 --- loss 0.8915483951568604
epoch 4 --- loss 0.6383162140846252
epoch 4 --- loss 0.6003917455673218
epoch 4 --- loss 0.6181979775428772
epoch 5 --- loss 0.6231785416603088
epoch 5 --- loss 0.7420096397399902
epoch 5 --- loss 0.6141404509544373
e

In [18]:
# Define a function to generate text given a prompt
def generate_text(prompt, model, tokenizer, max_length=100):
  # Encode the prompt to obtain input IDs and attention mask
  inputs = tokenizer.encode_plus(prompt, return_tensors="pt")
  # Extract input ids and attention mask
  input_ids = inputs['input_ids']
  attention_mask = inputs['attention_mask']

  # Generate text using the model
  outputs = model.generate(
      input_ids,                       # Provide input IDs to the model
      attention_mask=attention_mask,   # Provide attention mask to the model
      max_length=max_length            # Set the maximum length for the generated text
  )

  # Decode the generated text and return it, skipping special tokens
  return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [19]:
# Test the function
prompt = "In this research, we "
text_generated = generate_text(prompt, model, tokenizer, max_length = 500)
print(text_generated)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In this research, we   investigate the impact of reinforcement learning on the accuracy in tasks.
